In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import unicodedata
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait


In [3]:
#Scraping NBC news wild fires with Selenium and Beautifulsoup
def get_list_link_from_subject(subject, num_pages):


    PATH = "./chromedriver_win32/chromedriver.exe"
    driver = webdriver.Chrome(PATH)

    link_list = []

    driver.get("https://www.google.com/search?q="+subject+"&rlz=1C1CHBF_frFR863FR863&biw=1920&bih=880&sxsrf=AOaemvI0XcPZB9YWw9GUVGwWTEXPDVqRxQ:1638967714934&source=lnms&tbm=nws&sa=X&ved=2ahUKEwjGlsnDntT0AhWTTcAKHeyuDk4Q_AUoAXoECAEQAw")

    driver.find_element(By.XPATH, "//button[@class='VfPpkd-LgbsSe VfPpkd-LgbsSe-OWXEXe-k8QpJ VfPpkd-LgbsSe-OWXEXe-dgl2Hf nCP5yc AjY5Oe DuMIQc']").click() #accept google policy

    for i in range(num_pages):
        if i != 0:
            driver.find_element(By.ID, "pnnext").click()

        html_source = driver.page_source

        soup = BeautifulSoup(html_source, 'lxml')

        #Getting all g-card 
        g_card_list = soup.find_all("g-card")

        for g_card in g_card_list:
            a = g_card.find("a")
            link = a['href']
            link_list.append(link)

    driver.quit()

    print("Successfully scraped : ", len(link_list), " links")

    return link_list



In [4]:
#Now that we have the link list, for each link try to scrape the article if there is one and the date if there is one.
def get_df_from_link_list(link_list):

    my_timeout = 10

    data = []

    for i, link in enumerate(link_list):
        d = {}
        print(i+1, "/", len(link_list))

        try:
            html_text = requests.get(link, timeout=my_timeout).text

            soup = BeautifulSoup(html_text, 'lxml')

            title = soup.find('title')
            if title != None:
                d["Title"] = title.text

            d["Link"] = link
   
            time = soup.find('time')
            if time != None:
                if time.has_attr('datetime'):

                    d["Date"] = time['datetime']
            
            article = soup.find('article')
            if article != None:
                paragraphs = article.find_all('p')
                big_p = ""
                for p in paragraphs:
                    big_p = big_p + p.text + " "
                
                if big_p != "":
                    d["Content"] = unicodedata.normalize("NFKD", big_p).rstrip()
        except (requests.exceptions.Timeout, requests.exceptions.ConnectionError) as err: #Requests takes way too long
            print('Server taking too long. Try again later for page number' + str(i))

        data.append(d)

    return pd.DataFrame(data)

In [5]:
def get_df_from_subject(subject,num_pages):
    return get_df_from_link_list(get_list_link_from_subject(subject, num_pages))

In [11]:
my_scraped_data = get_df_from_subject("Wildfire",3)

C:\Users\SORLUC~1\AppData\Local\Temp/ipykernel_18928/477364689.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


Successfully scraped :  30  links
1 / 30
2 / 30
3 / 30
4 / 30
5 / 30
6 / 30
7 / 30
8 / 30
9 / 30
10 / 30
11 / 30
12 / 30
13 / 30
14 / 30
15 / 30
16 / 30
17 / 30
18 / 30
19 / 30
20 / 30
21 / 30
22 / 30
23 / 30
24 / 30
25 / 30
26 / 30
27 / 30
28 / 30
29 / 30
30 / 30


In [12]:
my_scraped_data

,Title,Link,Content,Date
0,USU Ecology Center speaker seeks to improve wi...,https://www.upr.org/utah-news/2021-12-08/usu-e...,In the face of increasing wildfire severity a...,NaN
1,Father and son accused of starting California'...,https://news.sky.com/story/father-and-son-accu...,NaN,NaN
2,Toxins in Wildfire Smoke May Make Their Way In...,https://consumer.healthday.com/b-12-9-toxins-i...,"THURSDAY, Dec. 9, 2021 (HealthDay News) -- Th...",NaN
3,Wildfire smoke may ramp up toxic ozone product...,https://www.sciencenews.org/article/wildfire-s...,Smoke from wildfires blazing in California bl...,2021-12-08T07:00:00-05:00
4,B.C. announces $1 million recovery grant for w...,https://globalnews.ca/news/8435230/b-c-announc...,The British Columbia government says it is im...,NaN
5,"After long delay, California lawmakers hold wi...",https://www.ijpr.org/wildfire/2021-12-08/after...,Lawmakers originally called for the hearing o...,NaN
6,"From Siberia to US west, wildfires spewed reco...",https://www.theguardian.com/world/2021/dec/07/...,Climate crisis fanned intense blazes that emi...,NaN
7,"Frosts, heatwaves and wildfires: the climate c...",https://www.theguardian.com/environment/2021/d...,"As the climate crisis intensifies, the wine i...",NaN
8,How prisons in a US state botched wildfire eva...,https://www.aljazeera.com/news/2021/12/8/how-p...,Is a conflict over water between farmers and ...,NaN
9,Wildfire dataset could help firefighters save ...,https://news.ucr.edu/articles/2021/12/08/wildf...,A team at UC Riverside led by computer scienc...,NaN


In [13]:
my_scraped_data.isna().sum()

Title       1
Link        0
Content    12
Date       24
dtype: int64

In [14]:
print("Titre " + my_scraped_data['Title'][0])
print("Lien : " + my_scraped_data['Link'][0])


print(my_scraped_data['Content'][0])

Titre USU Ecology Center speaker seeks to improve wildfire management |  UPR Utah Public Radio
Lien : https://www.upr.org/utah-news/2021-12-08/usu-ecology-center-speaker-seeks-to-improve-wildfire-management
 In the face of increasing wildfire severity across the US, one researcher speaking at USU this week hopes to bring back indigenous wildfire practices to current US wildfire management strategies that work to suppress fire. Current wildfire management practices miss the importance of fire on the landscape, said Christopher Roos, professor of anthropology at Southern Methodist University. He researches the interaction between society, climate and wildfire. “Smokey Bear made us believe that one of our options was living without wildfire. And, frankly, that's the only thing that is not an option for us...and we struggle, I think, partly because in Western society, we don't have a place for wildfire as anything other than a hazard,” Roos said. Roos’s work with the Jemez Pueblo and White